In [1]:
pip install spotipy python-dotenv



   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ---------------------------------------- 0/3 [redis]
   ------------- -------------------------- 1/3


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
    redirect_uri=os.getenv("SPOTIFY_REDIRECT_URI"),
    scope="user-read-recently-played user-top-read"
))

user = sp.current_user()
user["display_name"]


'neereafufi9'

EXPLORAR ARTISTAS FAVORITOS Y DESCUBRIR MÚSICA NUEVA
1. Pedir artistas favoritos.

In [6]:
favorite_artists = ['Galvan Real', 'Blake', 'Lola Índigo']

favorite_artists

['Galvan Real', 'Blake', 'Lola Índigo']

2. Buscar esos artistas en Spotify y obtener sus IDs

In [7]:
artist_ids = {}

for artist in favorite_artists:
    result = sp.search(q=artist, type="artist", limit=1)
    if result["artists"]["items"]:
        artist_id = result["artists"]["items"][0]["id"]
        artist_ids[artist] = artist_id

artist_ids


{'Galvan Real': '2iA56rE7fZUnD4GdsSUj3G',
 'Blake': '2FwMmxiCWQhEyO9k0nOyNF',
 'Lola Índigo': '3bvfu2KAve4lPHrhEFDZna'}

3. Obtener top 5 canciones de cada artista.

In [8]:
top_tracks = {}

for artist, artist_id in artist_ids.items():
    results = sp.artist_top_tracks(artist_id)
    tracks = [track["name"] for track in results["tracks"][:5]]
    top_tracks[artist] = tracks

top_tracks


{'Galvan Real': ['La Magia',
  'La Luna',
  'Amigos',
  'Volverte A Ver',
  'Deseo - Remix'],
 'Blake': ['Lengua de Serpiente',
  'El Diario del Loco',
  'Me Tiene Mal',
  'Ideales',
  'En Bucle'],
 'Lola Índigo': ['MOJA1TA',
  'EL TONTO',
  'LA REINA',
  '1000COSAS',
  'La Niña de la Escuela']}

In [13]:
artist_ids = {
    "Galvan Real": "2iA56rE7fZUnD4GdsSUj3G",
    "Blake": '2FwMmxiCWQhEyO9k0nOyNF',
    "Lola Indigo": '3bvfu2KAve4lPHrhEFDZna'
}

similar_artists = {}

for artist, artist_id in artist_ids.items():
    try:
        results = sp.artist_related_artists(artist_id)
        if results["artists"]:
            related = [a["name"] for a in results["artists"][:5]]
            similar_artists[artist] = related
        else:
            similar_artists[artist] = []
            print(f"⚠ No hay artistas relacionados para {artist}")
    except Exception as e:
        similar_artists[artist] = []
        print(f"⚠ Error con {artist}: {e}")

similar_artists


HTTP Error for GET to https://api.spotify.com/v1/artists/2iA56rE7fZUnD4GdsSUj3G/related-artists with Params: {} returned 404 due to Not Found
HTTP Error for GET to https://api.spotify.com/v1/artists/2FwMmxiCWQhEyO9k0nOyNF/related-artists with Params: {} returned 404 due to Not Found


⚠ Error con Galvan Real: http status: 404, code: -1 - https://api.spotify.com/v1/artists/2iA56rE7fZUnD4GdsSUj3G/related-artists:
 Not Found, reason: None
⚠ Error con Blake: http status: 404, code: -1 - https://api.spotify.com/v1/artists/2FwMmxiCWQhEyO9k0nOyNF/related-artists:
 Not Found, reason: None


HTTP Error for GET to https://api.spotify.com/v1/artists/3bvfu2KAve4lPHrhEFDZna/related-artists with Params: {} returned 404 due to Not Found


⚠ Error con Lola Indigo: http status: 404, code: -1 - https://api.spotify.com/v1/artists/3bvfu2KAve4lPHrhEFDZna/related-artists:
 Not Found, reason: None


{'Galvan Real': [], 'Blake': [], 'Lola Indigo': []}

4. Encontrar artistas similares (recomendaciones)

In [11]:
similar_artists = {}

for artist, artist_id in artist_ids.items():
    results = sp.artist_related_artists(artist_id)
    related = [a["name"] for a in results["artists"][:5]]
    similar_artists[artist] = related

similar_artists


HTTP Error for GET to https://api.spotify.com/v1/artists/2iA56rE7fZUnD4GdsSUj3G/related-artists with Params: {} returned 404 due to Not Found


SpotifyException: http status: 404, code: -1 - https://api.spotify.com/v1/artists/2iA56rE7fZUnD4GdsSUj3G/related-artists:
 Not Found, reason: None

Prepara la lista de URIs para la playlist.

In [15]:
track_uris = []

for artist, tracks in top_tracks.items():
    for track_name in tracks:
        result = sp.search(q=f"{track_name} {artist}", type="track", limit=1)
        items = result['tracks']['items']
        if items:
            track_uris.append(items[0]['uri'])


Crear una lista de canciones recomendadas para la playlist.

In [18]:
recommendation_tracks = []

for artist, artist_id in artist_ids.items():
    results = sp.artist_top_tracks(artist_id)
    for track in results["tracks"][:2]:
        recommendation_tracks.append(track["uri"])

recommendation_tracks


['spotify:track:1uw6eHdMtRlX6MtiZVJYB9',
 'spotify:track:06Ke9F6uDCPCRlryHzBegD',
 'spotify:track:1btXIMqhheMHFTSYRfYd7j',
 'spotify:track:0UO81pdmWKfhFi8F1VJfCl',
 'spotify:track:63rH77IJhmuQsNIxpCxaJ5',
 'spotify:track:4rKQ8daoK0aklVptea8A9E']

EXPLORAR PLAYLIST DESTACADAS
Obtener playlists destacadas de Spotify.

In [19]:
featured = sp.featured_playlists(limit=5)
featured_list = [(item["name"], item["id"]) for item in featured["playlists"]["items"]]

featured_list


HTTP Error for GET to https://api.spotify.com/v1/browse/featured-playlists with Params: {'locale': None, 'country': None, 'timestamp': None, 'limit': 5, 'offset': 0} returned 404 due to Not Found


SpotifyException: http status: 404, code: -1 - https://api.spotify.com/v1/browse/featured-playlists?limit=5&offset=0:
 Not Found, reason: None

Elegir una playlist y ver sus canciones.

In [20]:
playlist_id = featured_list[0][1]  # Primera playlist destacada

playlist_tracks = sp.playlist_tracks(playlist_id)

songs = [(t["track"]["name"], t["track"]["artists"][0]["name"]) 
         for t in playlist_tracks["items"]]

songs[:10]  # ver primeros 10


NameError: name 'featured_list' is not defined

In [ ]:
user_id = sp.current_user()["id"]

playlist = sp.user_playlist_create(user=user_id, name="Mi Playlist Descubierta", public=False)
sp.playlist_add_items(playlist["id"], recommendation_tracks)

print("✅ Playlist creada con tus descubrimientos!")
